# 최적의 하이퍼파라미터 찾는 방법

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import os
import random
from PIL import Image

import torch
import torchvision.transforms as T
import torchvision.models as models

from sklearn.metrics import f1_score
import optuna

In [2]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

device = 'mps'

In [3]:
annotations = pd.read_csv('data/annotations.csv')
annotations

,filename,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,toaster,sink,refrigerator,book,clock,vase,scissors,teddy bear,hair drier,toothbrush
0,000000289343.jpg,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000000061471.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000000472375.jpg,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000000520301.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000000579321.jpg,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4947,000000386134.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4948,000000097585.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4949,000000429530.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4950,000000031749.jpg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [4]:
len(annotations.columns)

81

In [5]:
annotations.sum()

filename      000000289343.jpg000000061471.jpg000000472375.j...
person                                                   2693.0
bicycle                                                   149.0
car                                                       535.0
motorcycle                                                159.0
                                    ...                        
vase                                                      137.0
scissors                                                   28.0
teddy bear                                                 94.0
hair drier                                                  9.0
toothbrush                                                 34.0
Length: 81, dtype: object

# Custom Dataset

In [6]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_path, annotations, transforms):
        super().__init__()
        self.data = []
        self.root_path = root_path
        self.transforms = transforms
        l = annotations.shape[0]

        for i in range(l):
            img = annotations.iloc[i, 0]
            classes = annotations.iloc[i, 1:].tolist()
            self.data.append((img, classes))

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_path, self.data[idx][0])
        img = Image.open(img_path).convert('RGB')
        img = self.transforms(img)
        label = torch.tensor(self.data[idx][1]).to(torch.float)

        return img, label

In [7]:
transforms = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])

total_dataset = CustomDataset('/Users/kimhongseok/cv_79_projects/part1/chapter3/6/data/images', annotations, transforms)

In [8]:
len(total_dataset[0][1])

80

In [9]:
classes = annotations.columns.tolist()[1:]
classes

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [10]:
total_num = len(total_dataset)
train_num, valid_num, test_num = int(total_num*0.8), int(total_num*0.1), int(total_num*0.1)
print(train_num, valid_num, test_num)

train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(total_dataset, [train_num+1, valid_num, test_num])

3961 495 495


In [11]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=100, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

# model

In [22]:
model = models.resnet50(pretrained=True)
model

/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(in_features=2048, out_features=len(classes))

for param in model.fc.parameters():
    param.requires_grad = True

# training, evaluation

### hyper parameter 설정

In [25]:
def training(model, train_dataloader, criterion, optimizer, threshold):
    model.train()
    train_loss = 0.0
    total_labels = []
    total_preds = []

    tbar = tqdm(train_dataloader)
    for images, labels in tbar:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        preds = (torch.sigmoid(outputs)>threshold).float()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        total_labels.extend(labels.cpu().numpy())
        total_preds.extend(preds.cpu().numpy())

    train_loss /= len(train_dataloader)
    train_f1 = f1_score(total_labels, total_preds, average='macro')

    return model, train_loss, train_f1

def evaluation(model, valid_dataloader, criterion, threshold):
    model.eval()
    valid_loss = 0.0
    total_labels = []
    total_preds = []

    with torch.no_grad():
        tbar = tqdm(valid_dataloader)
        for images, labels in tbar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            preds = (torch.sigmoid(outputs)>threshold).float()

            valid_loss += loss.item()
            total_labels.extend(labels.cpu().numpy())
            total_preds.extend(preds.cpu().numpy())

    valid_loss /= len(valid_dataloader)
    valid_f1 = f1_score(total_labels, total_preds, average='macro')

    return model, valid_loss, valid_f1

def training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, threshold, num_epochs):
    model.to(device)
    #train_loss_list = []
    #train_f1_list = []
    #valid_loss_list = []
    #valid_f1_list = []
    best_valid_f1 = float('inf')

    for epoch in range(num_epochs):
        model, train_loss, train_f1 = training(model, train_dataloader, criterion, optimizer, threshold)
        model, valid_loss, valid_f1 = evaluation(model, valid_dataloader, criterion, threshold)

        #train_loss_list.append(train_loss)
        #train_f1_list.append(train_f1)
        #valid_loss_list.append(valid_loss)
        #valid_f1_list.append(valid_f1)

        print(f'Train Loss: {train_loss}, Train F1: {train_f1}, Valid Loss: {valid_loss}, Valid F1: {valid_f1}')

    if best_valid_f1 < valid_f1:
        best_valid_f1 = valid_f1

    return best_valid_f1 #model, train_loss_list, train_f1_list, valid_loss_list, valid_f1_list

In [30]:
# 학습이 진행될 때 반복적으로 사용되는 함수 -> 하이퍼파라미터 범위, 모델, 학습 등이 정의되어 있다.
def objective(trial):
    lr = trial.suggest_float('learning_rate', 0.001, 0.005, log=True) # 학습률 범위
    num_epochs = trial.suggest_int('num_epochs', 1, 5) # 학습 반복 횟수 범위

    optimizer = torch.optim.Adam(model.parameters(), lr=lr) # 최적화 함수 -> lr이 조정된다.
    criterion = torch.nn.BCEWithLogitsLoss() # loss function
    
    valid_f1 = training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, 0.5, num_epochs) # 학습 진행 -> valid_f1을 반환 받는다.

    return valid_f1 # 학습 후 계산된 valid f1 값을 반환

In [31]:
study = optuna.create_study(direction='maximize') # study 객체 -> 반환받는 값을 최대, 최소화 하는 방향으로 학습 -> 현재는 최대화 하는 방향
study.optimize(objective, n_trials=10) # 최적의 파라미터를 찾는다. -> n_trials : n번의 학습을 진행하며 최적의 파라미터를 찾는다.

trial = study.best_trial

[I 2024-08-11 18:55:08,148] A new study created in memory with name: no-name-bf0e6b36-141b-48aa-8f26-4e6bacd1cfd9


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Train Loss: 0.17589944340288638, Train F1: 0.04061553271967392, Valid Loss: 0.10896392315626144, Valid F1: 0.06284278654776795


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


...

Train Loss: 0.05288559589534998, Train F1: 0.6713201052592637, Valid Loss: 0.08042533695697784, Valid F1: 0.45922900628972496


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


  0%|          | 0/40 [00:00<?, ?it/s]

[W 2024-08-11 19:04:25,282] Trial 5 failed with parameters: {'learning_rate': 0.0023141496611207876, 'num_epochs': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_43668/2028254700.py", line 8, in objective
    valid_f1 = training_loop(model, train_dataloader, valid_dataloader, criterion, optimizer, 0.5, num_epochs)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/zt/1l6rrzf977nbpmcz1d2j0vd40000gn/T/ipykernel_43668/2733620035.py", line 59, in training_loop
    model, train_loss, train_f1 = training(model, train_dataloader, criterion, optimizer, threshold)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

KeyboardInterrupt: 

In [ ]:
trial = study.best_trial